In [1]:
import pandas as pd 
import numpy as np 


In [2]:
err_df = pd.read_csv("err_content.csv",delimiter= ';') # read csv file of errors 
err_df.drop_duplicates(inplace=True)

In [4]:
# Return errors that contain keyword
def return_contain(str_ar,df):
    all_errors = []
    for str in str_ar:
        codes = df[df.CONTENT.str.contains(str)].CODE.unique() 
        for i in codes :
            df_ = df[df.CODE == i]
            for key,row in df_.iterrows():
                all_errors.append(row)
    return np.array(all_errors)

In [5]:
en_errors = return_contain(['encoder','Encoder','position','Position'],err_df)
en_errors_df = pd.DataFrame(en_errors)

over_errors = return_contain(['over','Over'],err_df)
over_errors_df = pd.DataFrame(over_errors)

servo_errors = return_contain(['Servo','servo'],err_df)
servo_errors_df = pd.DataFrame(servo_errors)


In [7]:
'''
Add Class column
0 encoder errors 
1 over errors 
2 servo errors 
'''
en_errors_df["CLASS"] = pd.Series(np.zeros(100000))
over_errors_df["CLASS"] = pd.Series(np.ones(100000))
servo_errors_df["CLASS"] = pd.Series(np.ones(100000)*2)

all_errors = pd.concat([en_errors_df,over_errors_df,servo_errors_df])


In [10]:
all_errors.sample(10,random_state=10)

,0,1,2,3,CLASS
287,46,22.10.2021,E0027,Encoder data change after stop [JT5Axis],0.0
788,349,13.11.2021,E0903,Servo communication stop,2.0
818,189,17.06.2021,E1339,[MC1]P-N high voltage.(Servo board1),2.0
97,328,4.08.2020,E3792,The encoder value changed in the power off. [J...,0.0
216,34,16.05.2022,E0898,Servo command stop [JT2Axis],2.0
578,221,3.08.2021,E0065,Over travel limit switch activated,1.0
96,328,4.08.2020,E3792,The encoder value changed in the power off. [J...,0.0
593,41,14.09.2020,E0038,??? [JT4?],1.0
963,277,6.02.2022,E0021,Abnormal servo tracking [JT1Axis],2.0
615,312,4.11.2021,E0898,Servo command stop [JT2Axis],2.0


In [58]:
no_error_df = pd.read_csv("final-no-error.csv",delimiter=';')
no_error_df.set_index('ROBO_ID',inplace=True)
no_error_df.drop(columns=['Unnamed: 0',"Unnamed: 0.1"],inplace=True)
no_error_df.REG_DATE = pd.to_datetime(no_error_df.REG_DATE)

In [59]:

from datetime import timedelta
'''
extract one day to make reg_date to occurrence data 
every data taken next day 
'''
no_error_df.REG_DATE = pd.to_datetime(no_error_df.REG_DATE - timedelta(1))



In [65]:
pd.to_datetime(all_errors[1]) 

0     2021-06-21
1     2021-07-13
2     2021-07-13
3     2021-07-13
4     2021-07-13
         ...    
991   2022-06-02
992   2022-05-30
993   2021-03-10
994   2021-01-31
995   2021-04-28
Name: 1, Length: 2229, dtype: datetime64[ns]

In [ ]:
iterate_over = no_error_df[pd.to_datetime(all_errors[1])  < no_error_df.REG_DATE.min()]
all_errors_ar = all_errors.to_numpy()
no_error__ar = no_error_df.to_numpy()

In [ ]:
error_period = []
for key,row in iterate_over.iterrows():
    date = row.OCCURRENCE
    period_4 = []
    for key2,row2 in iterate_over.iterrows():
        if(row2.REG_DATE >  date - timedelta(5) and row2.REG_DATE < date): 
            period_4.append(row2)
            print("yey")
    if(len(period_4) != 0):
        print(len(period_4))
        error_period.append(np.array(period_4))
